In [14]:
import requests as r
import json
import pandas as pd

# Example 1: Retrieve the 2024 F1 race schedule
url = 'http://ergast.com/api/f1/2024.json'
response = r.get(url)
schedule_json = json.loads(response.text)
schedule_str_json = json.dumps(schedule_json['MRData']['RaceTable']['Races'])
df_schedule = pd.read_json(schedule_str_json)
print("2024 F1 Race Schedule:\n", df_schedule)

# Example 2: Get race results for the first race of 2024

url = 'http://ergast.com/api/f1/2024/1/results.json'
response = r.get(url)
results_json = json.loads(response.text)

# Check if there are race results available
if results_json['MRData']['RaceTable']['Races']:
    results_str_json = json.dumps(results_json['MRData']['RaceTable']['Races'][0]['Results'])
    df_results = pd.read_json(results_str_json)
    print("Results for the first race of 2024:\n", df_results)
else:
    print("No race results available for the first race of 2024")

# Example 3: Fetch driver standings for the 2024 season
url = 'http://ergast.com/api/f1/2024/driverStandings.json'
response = r.get(url)
standings_json = json.loads(response.text)

# Check if there are standings available
if standings_json['MRData']['StandingsTable']['StandingsLists']:
    standings_str_json = json.dumps(standings_json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'])
    df_standings = pd.read_json(standings_str_json)
    print("Driver Standings for 2024:\n", df_standings)
else:
    print("No driver standings available for 2024")


/var/folders/7l/7tgw_8gd5dg1rrh9_ybll2ym0000gn/T/ipykernel_2734/757551252.py:10: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_schedule = pd.read_json(schedule_str_json)


2024 F1 Race Schedule:
     season  round                                                url  \
0     2024      1  https://en.wikipedia.org/wiki/2024_Bahrain_Gra...   
1     2024      2  https://en.wikipedia.org/wiki/2024_Saudi_Arabi...   
2     2024      3  https://en.wikipedia.org/wiki/2024_Australian_...   
3     2024      4  https://en.wikipedia.org/wiki/2024_Japanese_Gr...   
4     2024      5  https://en.wikipedia.org/wiki/2024_Chinese_Gra...   
5     2024      6  https://en.wikipedia.org/wiki/2024_Miami_Grand...   
6     2024      7  https://en.wikipedia.org/wiki/2024_Emilia_Roma...   
7     2024      8  https://en.wikipedia.org/wiki/2024_Monaco_Gran...   
8     2024      9  https://en.wikipedia.org/wiki/2024_Canadian_Gr...   
9     2024     10  https://en.wikipedia.org/wiki/2024_Spanish_Gra...   
10    2024     11  https://en.wikipedia.org/wiki/2024_Austrian_Gr...   
11    2024     12  https://en.wikipedia.org/wiki/2024_British_Gra...   
12    2024     13  https://en.wikipedia.

In [15]:
import requests as r
import json

def get_all_driver_records():
    base_url = 'http://ergast.com/api/f1/drivers.json'
    initial_response = r.get(base_url)
    initial_data = json.loads(initial_response.text)

    total_records = int(initial_data['MRData']['total'])
    limit = 1000  # Max records per request
    records_retrieved = 0
    all_drivers = []

    for offset in range(0, total_records, limit):
        url = f'{base_url}?limit={limit}&offset={offset}'
        response = r.get(url)
        data = json.loads(response.text)
        drivers = data['MRData']['DriverTable']['Drivers']
        all_drivers.extend(drivers)
        records_retrieved += len(drivers)

    return all_drivers, records_retrieved

# Retrieve all driver records
all_drivers, total_retrieved = get_all_driver_records()
print("Total number of driver records retrieved:", total_retrieved)


Total number of driver records retrieved: 858


In [17]:
import requests
import json
from datetime import datetime

def count_total_qualifying_results():
    total_records = 0
    current_year = datetime.now().year
    base_url = "http://ergast.com/api/f1/{year}/{round}/qualifying.json"

    for year in range(2003, current_year + 1):
        round = 1
        while True:
            url = base_url.format(year=year, round=round)
            response = requests.get(url)
            data = json.loads(response.text)

            race_results = data['MRData']['RaceTable']['Races']
            if not race_results:
                break  # No more races in this year

            total_records += len(race_results[0]['QualifyingResults'])
            round += 1

    return total_records

total_qualifying_results = count_total_qualifying_results()
print(f"Total number of qualifying results records: {total_qualifying_results}")



Total number of qualifying results records: 8467


In [18]:
import requests
import pandas as pd
from datetime import datetime

def get_qualifying_results():
    data_list = []
    current_year = datetime.now().year
    base_url = "http://ergast.com/api/f1/{year}/{round}/qualifying.json"

    for year in range(2003, current_year + 1):
        round = 1
        while True:
            url = base_url.format(year=year, round=round)
            response = requests.get(url)
            data = response.json()

            races = data['MRData']['RaceTable']['Races']
            if not races:  # No more races in this year
                break

            for race in races:
                for result in race.get('QualifyingResults', []):
                    data_list.append({
                        'Year': year,
                        'Round': round,
                        'RaceName': race['raceName'],
                        'CircuitId': race['Circuit']['circuitId'],
                        'DriverId': result['Driver']['driverId'],
                        'ConstructorId': result['Constructor']['constructorId'],
                        'Position': result['position'],
                        'Q1': result.get('Q1', ''),
                        'Q2': result.get('Q2', ''),
                        'Q3': result.get('Q3', '')
                    })

            round += 1

    return pd.DataFrame(data_list)

qualifying_results_df = get_qualifying_results()
qualifying_results_df.to_csv('qualifying_results.csv', index=False)
print("Qualifying results data saved to qualifying_results.csv")


Qualifying results data saved to qualifying_results.csv
